## Cleaned Data Ingestion to the MySql Database.

In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [2]:
dataset = pd.read_excel("cleaned_zomato_data.xlsx")
print(dataset.head(2))

   Order ID  Order Amount (INR)  Delivery Time (mins) Order Date  Rating  \
0         1                1930                    32 2023-04-30     4.6   
1         2                 371                    84 2023-01-06     2.7   

   Number of Items  Delivery Distance (km)  Delivery Agent ID Order Status  \
0                9                9.200017                995    cancelled   
1                1                8.175074                941    cancelled   

  Payment Method  ...  average_cost_for_two price_range  \
0     debit card  ...                   200           1   
1            upi  ...                  1000           3   

                                          highlights  aggregate_rating votes  \
0  ['credit card', 'delivery', 'takeaway availabl...               0.0     3   
1  ['cash', 'takeaway available', 'breakfast', 'n...               3.6    32   

  photo_count opentable_support delivery          state    area  
0           0               0.0        0         ke

In [3]:
try:
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="sa1tya@1005",
        database="cleaned_zomato_data"
    )
    if conn.is_connected():
        print("✅ Database Access Granted, Syncing all Data!!")
except Error as e:
        print(f'❌ Error : {e}')


✅ Database Access Granted, Syncing all Data!!


In [4]:
cursor = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS zomato_orders (
    `Order ID` VARCHAR(50),
    `Order Amount (INR)` FLOAT,
    `Delivery Time (mins)` INT,
    `Order Date` DATE,
    `Rating` FLOAT,
    `Number of Items` INT,
    `Delivery Distance (km)` FLOAT,
    `Delivery Agent ID` VARCHAR(50),
    `Order Status` VARCHAR(50),
    `Payment Method` VARCHAR(50),
    `Tip Amount (INR)` FLOAT,
    `Promo Code Applied` VARCHAR(10),
    `Has Free Delivery` VARCHAR(5),
    `res_id` VARCHAR(50),
    `name` VARCHAR(255),
    `type` VARCHAR(100),
    `address` TEXT,
    `city` VARCHAR(100),
    `locality` VARCHAR(100),
    `latitude` FLOAT,
    `longitude` FLOAT,
    `cuisines` TEXT,
    `timings` TEXT,
    `average_cost_for_two` INT,
    `price_range` INT,
    `highlights` TEXT,
    `aggregate_rating` FLOAT,
    `votes` INT,
    `photo_count` INT,
    `opentable_support` VARCHAR(5),
    `delivery` VARCHAR(5),
    `state` VARCHAR(100),
    `area` VARCHAR(100)
);

"""

cursor.execute(create_table_query)
conn.commit()

## Insert Data into the table 

In [5]:
for _, row in dataset.iterrows():
    # Convert timestamp to string format that MySQL can understand
    row_data = list(row)
    if isinstance(row_data[3], pd.Timestamp):  # Order Date is at index 3
        row_data[3] = row_data[3].strftime('%Y-%m-%d')
    
    insert_query = """
    INSERT INTO zomato_orders VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, tuple(row_data))

conn.commit()
print("✅ Data inserted successfully")


✅ Data inserted successfully


In [6]:
## DROP TABLE COMMAND
# cursor.execute("DROP TABLE IF EXISTS zomato_orders")

In [7]:
cursor.close()
conn.close()

## zomato data operations

In [8]:
try : 
    conn = mysql.connector.connect(
        host = "localhost",
        user = "root",
        password = "sa1tya@1005",
        database="cleaned_zomato_data"
    )

    if conn.is_connected():
        print("✅ Database Access Granted, Syncing all Data!!")
except Error as e:
        print(f'❌ Error : {e}')

✅ Database Access Granted, Syncing all Data!!


In [9]:
# cursor = conn.cursor()
Grouping_filtering_orders_by_time = pd.read_sql("SELECT MONTH(`Order Date`) AS order_month, COUNT(*) AS total_orders FROM zomato_orders GROUP BY MONTH(`Order Date`);",conn)
# cursor.execute(Grouping_filtering_orders_by_time,conn)

print(Grouping_filtering_orders_by_time)

    order_month  total_orders
0             4          2334
1             1          2346
2             2          2136
3             7          2468
4             6          2284
5             8          2266
6             5          2332
7             9          2386
8            10          1232
9            11          1048
10            3          2262
11           12          1064


C:\Users\hp\AppData\Local\Temp\ipykernel_9224\1105522004.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Grouping_filtering_orders_by_time = pd.read_sql("SELECT MONTH(`Order Date`) AS order_month, COUNT(*) AS total_orders FROM zomato_orders GROUP BY MONTH(`Order Date`);",conn)


## Finding top-rated restaurants

In [10]:
top_rated_restaurants = pd.read_sql("""SELECT `name`, AVG(`Rating`) AS avg_rating, COUNT(*) AS total_reviews
FROM zomato_orders
GROUP BY `name`
ORDER BY avg_rating DESC
LIMIT 10;"""
,conn)

print(top_rated_restaurants)

                   name  avg_rating  total_reviews
0        qabil-e-tareef         5.0              2
1      maa ambey sweets         5.0              2
2                 robot         5.0              2
3  manis chettinad mess         5.0              2
4             eatalio's         5.0              4
5     stream restaurant         5.0              2
6    lovely bake studio         5.0              2
7           desi dakaar         5.0              2
8         ghar ka khana         5.0              2
9                chaska         5.0              2


C:\Users\hp\AppData\Local\Temp\ipykernel_9224\2903537639.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_rated_restaurants = pd.read_sql("""SELECT `name`, AVG(`Rating`) AS avg_rating, COUNT(*) AS total_reviews


## Calculating average order value per city

In [11]:
average_order_per_city = pd.read_sql("""SELECT `city`, AVG(`Order Amount (INR)`) as avg_orders FROM zomato_orders GROUP BY `city` ORDER BY avg_orders DESC;""",conn)
print(average_order_per_city)

C:\Users\hp\AppData\Local\Temp\ipykernel_9224\3614253614.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  average_order_per_city = pd.read_sql("""SELECT `city`, AVG(`Order Amount (INR)`) as avg_orders FROM zomato_orders GROUP BY `city` ORDER BY avg_orders DESC;""",conn)


           city   avg_orders
0        howrah  2314.571429
1     panchkula  1794.678571
2   gandhinagar  1794.363636
3          agra  1784.623656
4         thane  1776.710843
..          ...          ...
71      vellore  1381.178082
72     siliguri  1376.773196
73    ghaziabad  1342.600000
74       guntur  1333.000000
75     srinagar  1324.900000

[76 rows x 2 columns]


## Which cities have the highest average tips?

In [12]:
highest_average_tips = pd.read_sql("""SELECT `city`, AVG(`Tip Amount (INR)`) as avg_tip
                                    FROM zomato_orders GROUP BY `city` 
                                    ORDER BY avg_tip DESC;
                                   """, conn)
print(highest_average_tips)

C:\Users\hp\AppData\Local\Temp\ipykernel_9224\4027289589.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  highest_average_tips = pd.read_sql("""SELECT `city`, AVG(`Tip Amount (INR)`) as avg_tip


            city     avg_tip
0          udupi  114.285714
1         shimla  113.571429
2        gwalior  110.612069
3      kharagpur  109.583333
4         nashik  108.342857
..           ...         ...
71      guwahati   83.745342
72  secunderabad   83.230769
73     faridabad   77.944444
74     ghaziabad   77.266667
75      junagadh   68.928571

[76 rows x 2 columns]


## Does free delivery impact average rating?

In [13]:
free_delivery_avg_rating = pd.read_sql("""
    SELECT `Has Free Delivery`, AVG(`Rating`) as avg_rating FROM zomato_orders GROUP BY `Has Free Delivery`;
""",conn)
print(free_delivery_avg_rating)

C:\Users\hp\AppData\Local\Temp\ipykernel_9224\2004066385.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  free_delivery_avg_rating = pd.read_sql("""


  Has Free Delivery  avg_rating
0                no    2.995925
1               yes    3.012797


## Which restaurant types are most popular in each state?

In [30]:
famous_restraunt_by_state = pd.read_sql("""SELECT `state`, `type`, COUNT(*) as total_orders
                                        FROM zomato_orders
                                        GROUP BY `state`, `type` 
                                        ORDER BY total_orders DESC ;""",
                                        conn)
print(famous_restraunt_by_state.head())

           state           type  total_orders
0    maharashtra  casual dining          1190
1    maharashtra    quick bites           892
2  uttar pradesh    quick bites           872
3     tamil nadu  casual dining           722
4  uttar pradesh  casual dining           530


C:\Users\hp\AppData\Local\Temp\ipykernel_9224\1263305965.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  famous_restraunt_by_state = pd.read_sql("""SELECT `state`, `type`, COUNT(*) as total_orders


## Is there a correlation between delivery distance and rating?

In [ ]:
relation_df = pd.read_sql("SELECT `Delivery Distance (km)`, `Rating` FROM zomato_orders", conn)
correlation = relation_df.corr()
print(correlation)

C:\Users\hp\AppData\Local\Temp\ipykernel_9224\1738916049.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  relation_df = pd.read_sql("SELECT `Delivery Distance (km)`, `Rating` FROM zomato_orders", conn)


                        Delivery Distance (km)    Rating
Delivery Distance (km)                1.000000  0.000809
Rating                                0.000809  1.000000


## Is there a correlation between delivery distance and rating?

In [42]:
relation_between_delivery_dist_and_rating = pd.read_sql("""
    SELECT `Delivery Distance (km)`, `Rating` FROM zomato_orders;""",conn)

print(relation_between_delivery_dist_and_rating)

C:\Users\hp\AppData\Local\Temp\ipykernel_9224\820359877.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  relation_between_delivery_dist_and_rating = pd.read_sql("""


       Delivery Distance (km)  Rating
0                     9.20002     4.6
1                     8.17507     2.7
2                    13.28040     3.9
3                    11.89290     1.9
4                     2.57048     3.4
...                       ...     ...
24153                 5.47461     2.8
24154                12.49950     4.4
24155                 1.00619     2.0
24156                 1.00619     2.0
24157                 2.76967     2.7

[24158 rows x 2 columns]


## What is the monthly growth in order volume?

In [50]:
monthly_growth = pd.read_sql("""
    SELECT DATE_FORMAT(`Order Date`,'%y-%m') as order_month, 
    COUNT(*) as total_orders
    FROM zomato_orders
    GROUP BY order_month
    ORDER BY order_month;
""",conn)

# print(monthly_growth)

C:\Users\hp\AppData\Local\Temp\ipykernel_9224\3175125549.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  monthly_growth = pd.read_sql("""


## Calculate the growth rate per month

In [ ]:
monthly_growth['growth_%'] = monthly_growth['total_orders'].pct_change() * 100
print(monthly_growth)

   order_month  total_orders   growth_%
0        23-01          1188        NaN
1        23-02          1006 -15.319865
2        23-03          1086   7.952286
3        23-04          1206  11.049724
4        23-05          1224   1.492537
5        23-06          1124  -8.169935
6        23-07          1232   9.608541
7        23-08          1140  -7.467532
8        23-09          1174   2.982456
9        23-10          1196   1.873935
10       23-11          1048 -12.374582
11       23-12          1064   1.526718
12       24-01          1158   8.834586
13       24-02          1130  -2.417962
14       24-03          1176   4.070796
15       24-04          1128  -4.081633
16       24-05          1108  -1.773050
17       24-06          1160   4.693141
18       24-07          1236   6.551724
19       24-08          1126  -8.899676
20       24-09          1212   7.637655
21       24-10            36 -97.029703
